In [1]:
import numpy as np
import sys
ROOT_PATH = './mylib/'
sys.path.append(ROOT_PATH)

random_seed = 19
np.random.seed(random_seed)

are_itersecting = lambda s, t: not (s['min_x'] > t['max_x'] | s['max_x'] < t['min_x'] | s['min_y'] > t['max_y'] | s['max_y'] < t['min_y'])

perturb = lambda x, pflip:((x != 0) ^ (np.rand() <= pflip))

def random_tile(n, m, min_width, max_width, min_height, max_height):
    x = np.random.randint(1, n)
    y = np.random.randint(1, m)
    w = np.random.randint(min_width, max_width)
    h = np.random.randint(min_height, max_height)
    return {'min_x':x, 'max_x':min(n, x + w), 'min_y':y, 'max_y':min(m, y + h)}

def generate_tiles(n, m, num_tiles, tile_width_range, tile_height_range, max_trials = 10, allow_overlap = True):
    tiles = []
    trials = 0
    while (len(tiles) < num_tiles) and (trials < max_trials):
        t = random_tile(n - 1, m - 1, tile_width_range[0], tile_width_range[1], tile_height_range[0], tile_height_range[1])
        if (allow_overlap or not any([are_itersecting(s, t) for s in tiles])):
            tiles.append(t)
            trials = 0
        else:
            trials = trials + 1
    return tiles

def generate_tiled_random_matrix(n, m, k, area_boundaries, noise, allow_overlap = True):
    tiles = generate_tiles(n, m, k, area_boundaries, area_boundaries, allow_overlap = allow_overlap)
    A = np.zeros((n, m))
    for t in tiles:
        A[t['min_x']:t['max_x'], t['min_y']:t['max_y']] = 1
    add_noise = lambda x, pflip: ((x != 0) | (np.random.rand() <= pflip))
    B = add_noise(A, noise)
    return B.astype(A.dtype), A

opts = {}

opts["n"] = 40
opts["m"] = 30
opts["k"] = 5
opts["minwidth"] = 5
opts["maxwidth"] = 10
opts["additivenoise"] = 0.1
opts["allowoverlap"] = True


A, org_A = generate_tiled_random_matrix(
        opts["n"],
        opts["m"],
        opts["k"],
        (opts["minwidth"], opts["maxwidth"]),
        opts["additivenoise"],
        allow_overlap = opts["allowoverlap"]
        )


In [2]:
from elbmf import ELBMF

k = 5
l1reg = 0.5
l2reg = 0.9
c = 1.05
maxiter = 100
tolerance = 0.001
beta = 0.0 # if beta > 0 -> use iPALM else use PALM

model = ELBMF(org_A, A, k, l1reg, l2reg, c, maxiter, tolerance, random_seed, beta)

model.factorize()

init_loss
recall : 0.7365853658536585, similarity : 0.6441666666666667, relative loss : 158465.0
result_loss
recall : 0.8975609756097561, similarity : 0.0175, relative loss : 4305.0
